###  This notebook will mainly be used for the coursera IBM capstone project

###### This next portion consists of section 1 of the assignment for week 3

In [1]:
import pandas as pd
import numpy as np
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
scraped_list = pd.read_html(url)
TOR_df = scraped_list[0]
notassigned_index = TOR_df[TOR_df['Borough'] == 'Not assigned'].index
TOR_df.drop(notassigned_index, inplace = True)
TOR_df = TOR_df.reset_index(drop = True)
TOR_df



,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### Gathering the Latitude and longitude:

In [2]:
#import geocoder
#lat_long_coords = None
#while (lat_long_coords is None):
#    g = geocoder.google('{}, Toronto, Ontario'.format(TOR_df['Postal Code']))
#    lat_long_coords = g.latlng

#latitude = lat_long_coords[0]
#longitude = lat_long_coords[1]

#due to the above code not working, I had to resolve to the solution that the assignment 

file = 'http://cocl.us/Geospatial_data'
lat_long = pd.read_csv(file)
lat_long.sort_values('Postal Code')
Toronto = pd.merge(TOR_df, lat_long, on='Postal Code')
Toronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [3]:
import json
!pip install geopy
from geopy.geocoders import Nominatim
!pip install folium==0.5.0
import folium

##### The following cell shows the Latitude and longitude of Toronto, which will come in handy when we decide to map the neighbourhoods and borough

In [4]:
geolocator = Nominatim(user_agent = 'to_explorer')
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude
print(" The Coordinates of Toronto are the following:", latitude,longitude)

 The Coordinates of Toronto are the following: 43.6534817 -79.3839347


##### The following Cell shows a map of Toronto with circled Postal Codes. When clicking on a circle marker, you will see the neighbourhood and borough of that Postal Code area

In [11]:
Toronto_Map = folium.Map(location = [latitude,longitude], zoom_start = 112)
for latitude, longitude, borough, neighbourhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood,borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup= label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_Map)
Toronto_Map

###### A quick analysis of our map, we can see that there is a large cluster of Neighborhoods and Boroughs where downtown Toronto lies. This seems reasonable due to the large number of addresses clustered in a tight area due to the highrise buildings and stores in such a tight knit area.